In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [2]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_AM.py

In [3]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekend.py

%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekend.py